# 作者を分類

# ライブラリのインストール

In [34]:
import urllib.request as req
import zipfile
import os.path
import MeCab
from gensim import models

# 設定

In [100]:
# Mecabの初期化
mecab = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
mecab.parse('')

'EOS\n'

In [101]:
# 保存したDoc2Vec学習済みモデルの読み込み
model = models.Doc2Vec.load('model/aozora.model')

# 関数の設定

In [102]:
# 分類用のZipファイルを開き、中の文書を取得する
def read_book(url, zipname, data_dir=''):
    if data_dir!='':
        data_dir = data_dir + '/'
    if not os.path.exists(data_dir + zipname):
        req.urlretrieve(url, data_dir + zipname)

    with zipfile.ZipFile(data_dir + zipname,"r") as zf:
        for filename in zf.namelist():
            with zf.open(filename,"r") as f:
                return f.read().decode("shift-jis")

In [103]:
# 引数のテキストを分かち書きして配列にする
def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

In [104]:
# 引数のタイトル、URLの作品を分類する
def similar(title, url):
    zipname = url.split("/")[-1]
        
    words = read_book(url, zipname, 'data')
    wakati_words = split_words(words)
    vector = model.infer_vector(wakati_words)
    print("--- 「" + title + '」 と似た作品は? ---')
    print(model.dv.most_similar([vector],topn=3))
    print("")

# most_similarの動作確認

In [105]:
model.dv['芥川 竜之介:羅生門'] # 羅生門の文書のベクトル

array([-0.34553424, -0.8099916 ,  0.1320717 , -0.36962444, -0.96730936,
        0.2026795 , -0.38726136, -0.02870343, -0.31768036,  0.43566397,
        0.142346  , -0.16096437, -0.6888628 , -0.21445431,  0.3017128 ,
       -0.866397  ,  0.43494555,  0.51943356,  0.7758687 , -0.11442843,
        0.6126681 , -0.25564638, -0.47085512,  1.326547  ,  0.09313752,
       -0.37290105, -0.45044604, -0.5181367 ,  0.36488143, -0.31015155,
       -0.40550023, -0.65483534,  0.2035963 ,  0.2374803 ,  0.54336673,
       -0.5641037 ,  1.0555903 , -0.13771537, -0.26282144,  0.7062503 ,
       -0.61628056,  0.17575487, -0.16973004,  0.4099476 , -0.42112097,
       -0.46529922, -0.1352025 , -0.8718371 ,  0.91479564,  0.45384002,
       -0.46268407,  0.38237268, -1.034633  , -0.3877343 , -0.39067718,
       -0.30435762, -0.15444154,  0.09377722, -0.29468405,  0.47438952,
        0.02679811,  0.46569148,  0.05961088,  0.53212893,  0.61433613,
        0.55960923,  0.33712873,  0.39602533, -0.44717494,  0.64

In [106]:
model.dv.most_similar('芥川 竜之介:羅生門') # model.dv.most_similar(tag) tagの書籍の類似度が高い他の書籍の確認できる。

[('芥川 竜之介:鼻', 0.8633846044540405),
 ('芥川 竜之介:老年', 0.7785667777061462),
 ('ポー エドガー・アラン:沈黙', 0.6819038987159729),
 ('宮澤 賢治:やまなし', 0.5962530970573425),
 ('芥川 竜之介:歯車', 0.5853080153465271),
 ('ポー エドガー・アラン:群集の人', 0.5389759540557861),
 ('ポー エドガー・アラン:落穴と振子', 0.4638517200946808),
 ('宮澤 賢治:どんぐりと山猫', 0.4256589114665985),
 ('ポー エドガー・アラン:黒猫', 0.42307889461517334),
 ('宮澤 賢治:セロ弾きのゴーシュ', 0.3974078893661499)]

In [107]:
model.dv.similarity('芥川 竜之介:羅生門', '芥川 竜之介:鼻') # model.dv.similarity(tag, tag) これでも同様にtag同士の書籍の文章の類似度を確認できる

0.8633846

In [108]:
wakati_practice = split_words('しかし、下人は雨がやんでも、格別どうしようと云う当てはない。ふだんなら、勿論、主人の家へ帰る可き筈である。所がその主人からは、四五日前に暇を出された。') # 文字列を分かち文字に変更
wakati_practice

['下人',
 '雨',
 'やむ',
 'しよう',
 '云う',
 'ふだん',
 '主人',
 '家',
 '帰る',
 '可',
 '筈',
 '所',
 '主人',
 '四',
 '五',
 '日',
 '前',
 '暇',
 '出す',
 'れる']

In [109]:
vector_practice = model.infer_vector(wakati_practice) # 文章を学習済みモデルのコーパスを利用してベクトル化
vector_practice

array([-0.049862  , -0.11872557, -0.01087023,  0.02856124, -0.08742945,
        0.00160917, -0.07838569,  0.02166463, -0.10686709,  0.06631034,
        0.06628606, -0.03881628, -0.01598293, -0.02965158,  0.02315019,
       -0.17809413,  0.17144728,  0.14320433,  0.0560545 , -0.0099434 ,
        0.00850975, -0.02274852, -0.09109015,  0.24712346,  0.01060093,
       -0.0378934 , -0.16818894, -0.13334571,  0.03340051, -0.15008353,
       -0.00722041, -0.01678697, -0.0254823 ,  0.0776235 ,  0.0748436 ,
       -0.08327407,  0.05070033, -0.08838551, -0.10204325,  0.12075105,
       -0.13309644, -0.00672207,  0.00991027, -0.01542001, -0.01978211,
       -0.19082315, -0.08912741, -0.08261994,  0.12059075,  0.05397246,
       -0.10999927,  0.12205971, -0.19815259, -0.09393897, -0.11225831,
       -0.03278235, -0.04998352,  0.08895844, -0.14998609,  0.13552794,
        0.07288243,  0.12522443, -0.01080731,  0.03435452,  0.14666848,
        0.10320993,  0.07496388,  0.04969525, -0.1057031 ,  0.09

In [110]:
model.dv.most_similar(vector_practice) # 指定した文章のベクトルを受け取ることで、その文章に近い書籍雨類似度順に表示

[('芥川 竜之介:老年', 0.8081603646278381),
 ('芥川 竜之介:羅生門', 0.7904042601585388),
 ('芥川 竜之介:鼻', 0.7562651634216309),
 ('宮澤 賢治:やまなし', 0.7369027137756348),
 ('ポー エドガー・アラン:沈黙', 0.7078360319137573),
 ('紫式部:源氏物語 03 空蝉', 0.681023895740509),
 ('芥川 竜之介:歯車', 0.6708187460899353),
 ('紫式部:源氏物語 05 若紫', 0.6295551061630249),
 ('宮澤 賢治:どんぐりと山猫', 0.628959596157074),
 ('紫式部:源氏物語 04 夕顔', 0.6014005541801453)]

# 予測

In [111]:
# 各作家の作品を１つずつ分類
similar("宮沢 賢治:よだかの星",
        "https://www.aozora.gr.jp/cards/000081/files/473_ruby_467.zip")

similar("芥川 龍之介:犬と笛",
        "https://www.aozora.gr.jp/cards/000879/files/56_ruby_845.zip")

similar("ポー エドガー・アラン:マリー・ロジェエの怪事件",
        "https://www.aozora.gr.jp/cards/000094/files/4261_ruby_54182.zip")

similar("紫式部:源氏物語 06 末摘花",
        "https://www.aozora.gr.jp/cards/000052/files/5021_ruby_11106.zip")

--- 「宮沢 賢治:よだかの星」 と似た作品は? ---
[('宮澤 賢治:やまなし', 0.8978421092033386), ('宮澤 賢治:セロ弾きのゴーシュ', 0.8691129088401794), ('芥川 竜之介:老年', 0.8077517747879028)]

--- 「芥川 龍之介:犬と笛」 と似た作品は? ---
[('芥川 竜之介:老年', 0.8206980228424072), ('芥川 竜之介:羅生門', 0.6174010634422302), ('芥川 竜之介:鼻', 0.6173392534255981)]

--- 「ポー エドガー・アラン:マリー・ロジェエの怪事件」 と似た作品は? ---
[('ポー エドガー・アラン:ウィリアム・ウィルスン', 0.8423877358436584), ('ポー エドガー・アラン:黒猫', 0.7621427178382874), ('ポー エドガー・アラン:落穴と振子', 0.684880256652832)]

--- 「紫式部:源氏物語 06 末摘花」 と似た作品は? ---
[('紫式部:源氏物語 05 若紫', 0.91771399974823), ('紫式部:源氏物語 03 空蝉', 0.9005005955696106), ('紫式部:源氏物語 02 帚木', 0.8392603397369385)]

